In [ ]:
from pathlib import Path
import os
from dotenv import load_dotenv

load_dotenv()

DB_PATH = Path(str(os.getenv('DB_PATH')))

import geopandas as gpd
import pandas as pd

document_files = pd.read_parquet(DB_PATH / "document_files.parquet")
locations = gpd.read_parquet(DB_PATH / "locations.parquet")
project_total_scopes = pd.read_parquet(DB_PATH / "project_total_scopes.parquet")
project_safety_scopes = pd.read_parquet(DB_PATH / "project_safety_scopes.parquet")
project_geos = gpd.read_parquet(DB_PATH / "project_geos.parquet")
projects = pd.read_parquet(DB_PATH / "projects.parquet")
location_year_files = pd.read_parquet(DB_PATH / "location_year_files.parquet")
document_collections = pd.read_parquet(DB_PATH / "document_collections.parquet")
location_to_project = pd.read_parquet(DB_PATH / "_location_to_project.parquet")
imgs_to_filesAPI = pd.read_parquet(DB_PATH / '_imgs_to_filesAPI.parquet')
docs_to_filesAPI = pd.read_parquet(DB_PATH / '_docs_to_filesAPI.parquet')
image_pairs = pd.read_parquet(DB_PATH / 'processed' / 'image_pairs.parquet')

# Image Change Identifier

In [19]:
from st_preprocessing.config import DATA_PATH
from streettransformer.llms.models.queries import QUERIES

queries123_sample = pd.read_parquet(DB_PATH / 'groundtruth' / 'change_identifier_classes.parquet')

queries123_sample.drop_duplicates(subset=['location_id','year_start','year_end'])
queries123_sample.drop_duplicates()#['#subset=['location_id','year_start','year_end', ])


# Ok so like 4 of them have projects with different scopes at the same time :/
# image_pairs_oai_export
# # image_pairs_oai_export['image_A'] = [DATA_PATH / x for x in image_pairs_oai_export['image_A']]
# # image_pairs_oai_export['image_B'] = [DATA_PATH / x for x in image_pairs_oai_export['image_B']]


# image_pairs = image_pairs.merge(
#     locations[['location_id', 'sample']].rename(columns={'sample': 'sample_source'}),
#     left_on='location_id', right_on='location_id',
# )
queries123_sample_merged = queries123_sample.merge(
    image_pairs, 
    left_on=['location_id', 'year_start','year_end'], right_on=['location_id','year_start','year_end'],
    how='left', indicator=True
)

queries123_sample_merged['_merge'].value_counts() # Good: all 2000
with open('../data_oai_files/query123_top1000_sample.csv', 'w') as f:
    f.write('item_id,prompt,file_labels\n')
    for row in queries123_sample_merged.head(1000).itertuples():
        key         = f'"{row.location_id}-{row.year_start}-{row.year_end}"'
        query       = '"' + QUERIES["image_change_identifier"].text() + '"'
        file_labels = f'"before:{row.imagery_path_start};after:{row.imagery_path_end}"'
        f.write(f'{key},{query},{file_labels}\n')

# # image_pairs_oai_export = pd.DataFrame()
# # del image_pairs_oai_export
# # #image_pairs['imagery_path_start']

# Read back in results

In [85]:
import json
change_identifier_top100_output = pd.read_json('../data_oai_files/query1_top1000_output-4o.csv',lines=True)
DB_PATH

ID_COLS = ['location_id', 'year_start', 'year_end']
RESULTS_COLS = ['change_detected','confidence','features']
change_identifier_top100_output[ID_COLS] = change_identifier_top100_output['item_id'].str.split('-', expand=True)
change_identifier_top100_output['location_id'] = change_identifier_top100_output['location_id'].astype(int)

#validation_df = change_identifier_top100_output[['location_id', 'year_start', 'year_end']].copy()
#validation_df

change_identifier_top100_output[RESULTS_COLS] = change_identifier_top100_output['output_text'].apply(pd.Series)[['change_detected','confidence','features']]
validation_df = change_identifier_top100_output[ID_COLS + RESULTS_COLS]

(
    validation_df.merge(
        queries123_sample,
        left_on = ID_COLS, right_on=ID_COLS
    ).groupby('change_identifier_safety')['change_detected']
    .value_counts(normalize=True)
    .unstack()
    .rename(columns={True: 'Change Detected', False: 'Change Not Detected'})
    .T
    .style
        .background_gradient(cmap='Greens')
        .format('{:.1%}')
)

change_identifier_safety,Change Between,Change Out of Range,No Change
change_detected,,,
Change Not Detected,3.2%,6.0%,3.4%
Change Detected,96.8%,94.0%,96.6%
